In [ ]:
# GUI for realtime
import sys
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui, QtCore
%gui qt

# Plot graph in notebook
import matplotlib.pyplot as plt

# Processing
from cmath import exp, pi # cmath for complex number 
import numpy as np
import pyaudio

# Performance
import time

# Checking fft and stft
import scipy.signal as signal
from scipy.fftpack import fft

## 1. Generate signal 

In [ ]:
# unitstep function
def rect_func(x, k):
    res = []
    N = len(x)
    for i in x:
        if i <= k:
            res.append(0)
        else:
            res.append(1)
    return np.array(res)

In [ ]:
# Generate signal 
fs = 1024                      # sampling rate
t = np.arange(0, 2, 1/fs)      # time 
# Signal
x = np.sin(2*pi*10*t) + \
    np.sin(2*pi*20*t)*rect_func(t, 0.3) + \
    np.sin(2*pi*50*t)*rect_func(t, 0.6) + \
    np.sin(2*pi*100*t)*rect_func(t, 1) + \
    np.sin(2*pi*200*t)*rect_func(t, 1.3) + \
    np.sin(2*pi*400*t)*rect_func(t, 1.6)
# Plot signal in time domains
plt.plot(t, x)
plt.title("Waveform")
plt.xlabel('Time(s)')
plt.ylabel('Amplitude')

## 2. FFT and Spectrum

In [ ]:
# N muse be a 2**n integers 
def compute_fft(x):
    N = len(x)
    if N <= 1:
        return x
    even = compute_fft(x[0::2])
    odd = compute_fft(x[1::2])
    
    T = [exp(-2j*pi*k/N)*odd[k] for k in range(N//2)]
    return np.array([even[k] + T[k] for k in range(N//2)] + [even[k] - T[k] for k in range(N//2)])

In [ ]:
# Plot spectrum of the signal using compute_fft function
n = len(t)
# Frequency axis - max frequency is fs/2 (Nyquist rate)
# n/2 because only need 1-sided spectrum
freqs = (fs/2)*np.linspace(0, 1, n/2)
X = compute_fft(x)
X_m = np.array(X[0:int(n/2)])
# Plot signal in frequency domain
plt.plot(freqs, (2/n)*abs(X_m))
plt.title('Spectrum')
plt.xlabel('Frequency(Hz)')
plt.ylabel('Magnitude')

## 3. STFT and Spectrogram

In [ ]:
# frameSamples must be a 2^n integers - FFT alogirthms
def compute_stft(x, frameSamples, newSamples):
    X = []
    # number of windows
    oneside = int(frameSamples/2) +1
    k = int((len(x) - frameSamples)/(frameSamples - newSamples))+1
    for i in range(0, k):
        start = i*(frameSamples-newSamples)
        X.append(compute_fft(x[start:start+frameSamples])[0:oneside])
    X = np.array(X).T
    return X/frameSamples

In [ ]:
# Plot spectrogram using stft
Y = compute_stft(x, 128, 64)
shapeY = Y.shape
# print(Y.shape)
freqs = np.linspace(0, fs/2, shapeY[0])
# print(len(freqs))
tx = np.linspace(0, len(x)/fs, shapeY[1]) 
plt.pcolormesh(tx, freqs, np.abs(Y))
plt.title('Spectrogram')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')

## 4. Waveform and Spectrum of voice in real-time

### a. Read and convert voice data

In [ ]:
CHUNK = 1024              # number of samples for each time analyze
FORMAT = pyaudio.paInt16  # 2 bytes per sample
CHANNELS = 1              # single channel (left)
RATE = 44100              # sampling rate
STFT_FFT = 128            # number of points using for stft
N_OVERLAP = 64             # number of points overlap in stft

In [ ]:
# Stream a buffer of voice data from microphone
voice = pyaudio.PyAudio()
stream = voice.open(format = FORMAT,
                    channels = CHANNELS,
                    rate = RATE,
                    input = True,
                    output = True,
                    frames_per_buffer = CHUNK)

In [2]:
%run -i signal_and_spectrum.py

In [1]:
%run -i spectrogram.py

spectrogram.py:25: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  y = np.fromstring(data, 'int16')
